<a href="https://colab.research.google.com/github/doduck210/Instrument_Classifier/blob/main/instrument_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#for using CoLab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#for using CoLab
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [3]:
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import random
torch.cuda.get_device_name(0)

'Tesla T4'

In [4]:
#데이터 확인
npz=np.load('/content/gdrive/My Drive/audio/cqt.npz')
x=npz['spec']
y=npz['instr']
print("input shape : ",x.shape) 
print("label shape : ",y.shape)

input shape :  (20488, 168, 87)
label shape :  (20488, 2)


In [13]:
use_cuda = torch.cuda.is_available()
class InstrumentClassifier(nn.Module):
  def __init__(self):
    super(InstrumentClassifier,self).__init__()
    #input : [256, 1, 168, 87]
    conv1=nn.Conv2d(1,10,(23,4)) #6@146x84
    bn1=nn.BatchNorm2d(10)
    #ReLU
    pool1=nn.MaxPool2d(2) #6@73x42
    conv2=nn.Conv2d(10,3,(6,3)) #10@68x40
    bn2=nn.BatchNorm2d(3)
    #ReLU
    pool2=nn.MaxPool2d(2) #10@34x20
    conv3=nn.Conv2d(16,32,(5,5)) #28@30x16
    bn3=nn.BatchNorm2d(32)
    #ReLU
    pool3=nn.MaxPool2d(2) #28@15x8
    
    dropout= nn.Dropout()
    #nn.init.xavier_uniform_(conv1.weight)
    #nn.init.xavier_uniform_(conv2.weight)
    #nn.init.xavier_uniform_(conv3.weight)

    self.conv_module=nn.Sequential(
        conv1, bn1,
        nn.ReLU(),
        pool1,
        conv2, bn2,
        nn.ReLU(),
        pool2#,
        #conv3,
        #nn.ReLU(),# dropout,
        #pool3
    )

    fc1=nn.Linear(3*34*20,174)
    #ReLU
    #fc2=nn.Linear(500,174)
    
    self.fc_module=nn.Sequential(
        fc1
        #nn.ReLU(),
        #fc2
    )
    if use_cuda:
      self.conv_module=self.conv_module.cuda()
      self.fc_module=self.fc_module.cuda()
    
  def forward(self,x):
    out=self.conv_module(x)
    dim=1
    out=out.view(out.size(0),-1)
    out=self.fc_module(out)
    return F.softmax(out,dim=1)

In [14]:
cnn=InstrumentClassifier()
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(cnn.parameters(),lr=1e-3)

In [15]:
ridx = list(range(len(x)))
random.shuffle(ridx)

test_size=len(ridx)//5 # train:test=8:2
test_ridx, train_ridx=ridx[:test_size],ridx[test_size:]
batch_size = 256

for e in range(100):
  ac=0
  for b in range(0,len(train_ridx),batch_size) : 
    tr=train_ridx[b:b+batch_size]
    input=torch.tensor(x[tr])
    input=input.unsqueeze(1)
    input=input.cuda()
    output=torch.tensor(y[tr,0])
    output=torch.tensor(output,dtype=torch.long)
    output=output.cuda()
    #print("x : ",input.shape) #torch.Size([256, 1, 168, 87])
    #print("y : ",output.shape) #torch.Size([256])

    optimizer.zero_grad()
    model_output=cnn(input) #torch.Size([256, 174])
    #print(model_output.shape)
    loss=criterion(model_output,output)
    loss.backward()
    optimizer.step()
    for acc in range(0,model_output.shape[0]) : 
      pre=torch.argmax(model_output[acc])
      if pre==output[acc]: ac+=1;
    
    if (b//batch_size)%10==0 : print('e : ',e,"loss : ",loss)
  print("train acc : ",ac/len(train_ridx))

  test_ac=0
  for b in range(0,len(test_ridx),batch_size) : 
    tr=test_ridx[b:b+batch_size]
    input=torch.tensor(x[tr])
    input=input.unsqueeze(1)
    input=input.cuda()
    output=torch.tensor(y[tr,0])
    output=torch.tensor(output,dtype=torch.long)
    output=output.cuda()
    model_output=cnn(input)
    for acc in range(0,model_output.shape[0]):
      pre=torch.argmax(model_output[acc])
      if pre==output[acc]: test_ac+=1;

  print("test acc : ",test_ac/len(test_ridx))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


e :  0 loss :  tensor(5.1588, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1479, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1239, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.0969, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.0308, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.0298, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(4.9845, device='cuda:0', grad_fn=<NllLossBackward>)
train acc :  0.10981636263803307


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


test acc :  0.22211374176226506
e :  1 loss :  tensor(4.9296, device='cuda:0', grad_fn=<NllLossBackward>)
e :  1 loss :  tensor(4.9422, device='cuda:0', grad_fn=<NllLossBackward>)
e :  1 loss :  tensor(4.9494, device='cuda:0', grad_fn=<NllLossBackward>)
e :  1 loss :  tensor(4.9259, device='cuda:0', grad_fn=<NllLossBackward>)
e :  1 loss :  tensor(4.8689, device='cuda:0', grad_fn=<NllLossBackward>)
e :  1 loss :  tensor(4.8657, device='cuda:0', grad_fn=<NllLossBackward>)
e :  1 loss :  tensor(4.8009, device='cuda:0', grad_fn=<NllLossBackward>)
train acc :  0.3220669879812092
test acc :  0.36612155235538196
e :  2 loss :  tensor(4.7777, device='cuda:0', grad_fn=<NllLossBackward>)
e :  2 loss :  tensor(4.7676, device='cuda:0', grad_fn=<NllLossBackward>)
e :  2 loss :  tensor(4.8034, device='cuda:0', grad_fn=<NllLossBackward>)
e :  2 loss :  tensor(4.8067, device='cuda:0', grad_fn=<NllLossBackward>)
e :  2 loss :  tensor(4.7615, device='cuda:0', grad_fn=<NllLossBackward>)
e :  2 loss :  t